<a href="https://colab.research.google.com/github/vahedshaik/cmpe-297-project/blob/main/Classification_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training, Hyperparameter tuning and deploying a PyTorch BERT based Model on Vertex AI for Emotion Classification

## Dataset:
Emotion from Hugging Face Datasets:
We have used this dataset for emotion classification by fine tuning the pretrained BERT model. The dataset contains labels belonging to multiple classes. It is a dataset of Twitter messages with six basic emotions: anger, fear, joy, love, sadness, and surprise

## Set up the local environment for development

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
!pip -q install {USER_FLAG} --upgrade transformers
!pip -q install {USER_FLAG} --upgrade datasets
!pip -q install {USER_FLAG} --upgrade tqdm
!pip -q install {USER_FLAG} --upgrade cloudml-hypertune

## Installing the Vertex SDK for Python

In [ ]:
!pip -q install {USER_FLAG} --upgrade google-cloud-aiplatform

## **Need to restart the kernel here

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Setting up the project details

In [ ]:
PROJECT_ID = "123456"  # <---CHANGE THIS TO YOUR PROJECT

import os

# Get your Google Cloud project ID using google.auth
if not os.getenv("IS_TESTING"):
    import google.auth

    _, PROJECT_ID = google.auth.default()
    print("Project ID: ", PROJECT_ID)

# validate PROJECT_ID
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "123456":
    print(
        f"Please set your project id before proceeding to next step. Currently it's set as {PROJECT_ID}"
    )

Project ID:  starry-lens-333804


In [ ]:
from datetime import datetime


def get_timestamp():
    return datetime.now().strftime("%Y%m%d%H%M%S")


TIMESTAMP = get_timestamp()
print(f"TIMESTAMP = {TIMESTAMP}")


TIMESTAMP = 20211209030542


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOG_CLD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOG_CLD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

## Create a Cloud Storage bucket

The cloud storage bucket helps in storing the model artifacts and hyperparameter tuning results. While creating and deploying the model on Vertex AI this is utilized. We created a cloud bucket by executing the following set of commands on the notebook instance.

In [ ]:
BCT_NAME = "gs://[your-bucket-name]"
REGION = "us-west1"

In [ ]:
if BCT_NAME == "" or BCT_NAME is None or BCT_NAME == "gs://[your-bucket-name]":
    BCT_NAME = f"gs://{PROJECT_ID}aip-{get_timestamp()}"

In [ ]:
print(f"PROJECT_ID = {PROJECT_ID}")
print(f"BUCKET_NAME = {BCT_NAME}")
print(f"REGION = {REGION}")

In [ ]:
! gsutil mb -l $REGION $BCT_NAME

In [ ]:
! gsutil ls -al $BCT_NAME

## Importing the required libraries

In [ ]:
import base64
import json
import os
import random
import sys

import google.auth
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip
from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google.protobuf.json_format import MessageToDict

In [ ]:
from IPython.display import HTML, display

In [ ]:
import datasets
import numpy as np
import pandas as pd
import torch
import transformers
from datasets import ClassLabel, Sequence, load_dataset
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          EvalPrediction, Trainer, TrainingArguments,
                          default_data_collator)

In [ ]:
print(f"Notebook runtime: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print(f"PyTorch version : {torch.__version__}")
print(f"Transformers version : {datasets.__version__}")
print(f"Datasets version : {transformers.__version__}")

Notebook runtime: GPU
PyTorch version : 1.9.0
Transformers version : 1.16.1
Datasets version : 4.12.5


In [ ]:
APP_NAME = "finetuned-bert"

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
##Downloading and loading the emotion dataset from huggingface
datasets = load_dataset("emotion")

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/204k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/207k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset emotion downloaded and prepared to /home/jupyter/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
##Printing the dataset to see the structure
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
##Dataset shape
print(
    "Total #  number of rows in training dataset {} and size {:5.2f} MB".format(
        datasets["train"].shape[0], datasets["train"].size_in_bytes / (1024 * 1024)
    )
)
print(
    "Total # number of rows in test dataset {} and size {:5.2f} MB".format(
        datasets["test"].shape[0], datasets["test"].size_in_bytes / (1024 * 1024)
    )
)

Total # of rows in training dataset 16000 and size  4.05 MB
Total # of rows in test dataset 2000 and size  4.05 MB


In [ ]:
datasets["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [ ]:
##Labels
lbl= datasets["train"].unique("label")
lbl

[0, 3, 2, 5, 4, 1]

In [ ]:
def show_random_elements(dataset, num_exms=2):
    assert num_exms <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    pck = []
    for _ in range(num_exms):
        i = random.randint(0, len(dataset) - 1)
        while i in pck:
            i = random.randint(0, len(dataset) - 1)
        pck.append(i)

    dt = pd.DataFrame(dataset[pck])
    for columns, tp in dataset.features.items():
        if isinstance(tp, ClassLabel):
            dt[columns] = dt[columns].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(tp.feature, ClassLabel):
            dt[columns] = dt[columns].transform(
                lambda x: [tp.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])


,text,label
0,i was feeling brave and wanted to try my hand at free motion quilting,joy
1,i could feel it but it didnt hurt,sadness


## Data Preprocessing
We used the ‘Tokenizer’ class from Hugging Face Transformers to tokenize and preprocess the input data to the format required by the model. The below screenshot represents a sample of the tokens created for input text:

In [ ]:
batch_size = 16
max_seq_length = 128
model_name_or_path = "bert-base-cased"

In [ ]:
tknzr = AutoTokenizer.from_pretrained(
    model_name_or_path,
    use_fast=True,
)

In [ ]:
tknzr("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
##Results of tokenizing
tknzr("Hello and good morning!")

{'input_ids': [101, 8667, 1105, 1363, 2106, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
##Results of tokenizing
acp = datasets["train"][4]
print(acp)

{'text': "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .", 'label': 1}


In [ ]:
##Results of tokenizing
tknzr(
    ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],
    is_split_into_words=True,
)

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 3325, 1154, 1734, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# Dataset loading repeated here to make this cell idempotent
# Since we are over-writing datasets variable
dts = load_dataset("emotion")




def preprocess_function(examples):
    """
    Tokenize the input example texts
    NOTE: The same preprocessing step(s) will be applied
    at the time of inference as well.
    """
    args = (examples["text"],)
    result = tokenizer(
        *args, padding="max_length", max_length=max_seq_length, truncation=True
    )


    return result


# apply preprocessing function to input examples
dts = dts.map(preprocess_function, batched=True, load_from_cache_file=True)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## Model fine-tuning
Transfer learning is a technique where a deep learning model trained on a large dataset is used to perform similar tasks on another dataset. This is done by freezing the middle layers while modifying the input and output layers that are suitable for our task.

In this study we are finetuning the BERT model which is pre-trained on a large corpus of unlabeled text including the entire Wikipedia (2,500 million words) and Book Corpus (800 million words). The advantage of using BERT includes only having to slightly tune it for the emotion classification task. This also results in quicker development and uses a much smaller dataset.


In [ ]:
mdl = AutoModelForSequenceClassificationSequenceClassification.from_pretrained(
    model_name_or_path, num_labels=len(label_list)
)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

We create a trainer object with all the required parameters including learning rate, number of epochs, weight decay and batch size for training. The below screenshots show the same:

In [ ]:
args = TrainingArguments(
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    output_dir="/tmp/cls",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def cpt_mtrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["test"],
    data_collator=default_data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
history_train = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 16000
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.176300,0.273056,0.925000
2,0.133700,0.206070,0.921000
3,0.103600,0.247529,0.926000
4,0.079000,0.321118,0.924000
5,0.052000,0.421277,0.924500
6,0.039600,0.410756,0.926500
7,0.021200,0.451202,0.921500
8,0.020100,0.447655,0.926000
9,0.013600,0.476519,0.920000
10,0.008700,0.485455,0.921000


Saving model checkpoint to /tmp/cls/checkpoint-500
Configuration saved in /tmp/cls/checkpoint-500/config.json
Model weights saved in /tmp/cls/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /tmp/cls/checkpoint-500/tokenizer_config.json
Special tokens file saved in /tmp/cls/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /tmp/cls/checkpoint-1000
Configuration saved in /tmp/cls/checkpoint-1000/config.json
Model weights saved in /tmp/cls/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /tmp/cls/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /tmp/cls/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to /tmp/cls/checkpoint-1500
Configuration saved in /tmp/cls/checkpoint-1500/config.json
M

In [ ]:
saved_model_path = "./models"
!mkdir ./models

mkdir: cannot create directory ‘./models’: File exists


In [ ]:
trainer.save_model(saved_model_path)

Saving model checkpoint to ./models
Configuration saved in ./models/config.json
Model weights saved in ./models/pytorch_model.bin
tokenizer config file saved in ./models/tokenizer_config.json
Special tokens file saved in ./models/special_tokens_map.json


In [ ]:
hstry = trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16


In [ ]:
!pip install tensorboard

     |████████████████████████████████| 5.8 MB 7.8 MB/s            
     |████████████████████████████████| 781 kB 42.0 MB/s            
     |████████████████████████████████| 4.9 MB 79.5 MB/s            
     |████████████████████████████████| 288 kB 82.5 MB/s            
     |████████████████████████████████| 126 kB 78.2 MB/s            


In [ ]:
from torch.utils.tensorboard import SummaryWriter
ts = SummaryWriter()

In [ ]:
history_train

TrainOutput(global_step=10000, training_loss=0.06252609403133393, metrics={'train_runtime': 1407.4847, 'train_samples_per_second': 113.678, 'train_steps_per_second': 7.105, 'total_flos': 1.052482019328e+16, 'train_loss': 0.06252609403133393, 'epoch': 10.0})

## Running predictions locally

In [ ]:
ts.add_scalar("Evalutation Loss", history['eval_loss'], history['epoch'])

In [ ]:
model_name_or_path = "bert-base-cased"
label_text = {0: "Sadness", 1: "Joy", 2:"love", 3:"anger", 4:"fear"}
saved_modelpath = saved_model_path


def predict(input_text, saved_model_path):
    # initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

    # preprocess and encode input text
    tokenizer_args = (input_text,)
    predict_input = tokenizer(
        *tokenizer_args,
        padding="max_length",
        max_length=128,
        truncation=True,
        return_tensors="pt",
    )

    # load trained model
    loaded_model = AutoModelForSequenceClassification.from_pretrained(saved_model_path)

    # get predictions
    output = loaded_model(predict_input["input_ids"])

    # return labels
    label_id = torch.argmax(*output.to_tuple(), dim=1)

    print(f"Review text: {input_text}")
    print(f"Sentiment : {label_text[label_id.item()]}\n")

In [ ]:
# example #1
rvw_text = (
    "im feeling quite sad and sorry for myself but ill snap out of it soon"
)
predict_input = predict(rvw_text, saved_model_path)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/jupyter/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /home/jup

Review text: im feeling quite sad and sorry for myself but ill snap out of it soon
Sentiment : Sadness



## Training on Vertex AI

For larger datasets and models as in our case, building a training pipeline by leveraging Vertex AI is the most effective method. The training job on Vertex AI is carried out by packaging the code and creating a training pipeline to orchestrate a training job. The 3 steps we have followed are:

• Packaging the training code as a Python source distribution

• Hyperparameter training job

• Finally running the training job

### Running Custom Job on Vertex AI with a pre-built container

We are using pre-built container for PyTorch and packaging the training application code by adding the following Python dependencies - transformers, datasets and tqdm - in the setup.py file

In [ ]:
TRN_IMAGE = (
    "us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-7:latest"
)

APP_DIR = "python_package"

SRC_PCKG = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"
python_package_gcs_uri = (
    f"{BUCKET_NAME}/pytorch-on-gcp/{APP_NAME}/train/python_package/trainer-0.1.tar.gz"
)
pyt_NAME = "trainer.task"

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/setup.py

from setuptools import find_packages
from setuptools import setup
import setuptools

from distutils.command.build import build as _build
import subprocess


REQUIRED_PACKAGES = [
    'transformers',
    'datasets',
    'tqdm',
    'cloudml-hypertune'
]

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Vertex AI | Training | PyTorch | Text Classification | Python Package'
)

Overwriting ./python_package/setup.py


### Running the custom training job locally to ensure no errors

In [ ]:
!cd {APP_DIR} && python3 setup.py sdist --formats=gztar

running sdist
running egg_info
creating trainer.egg-info
writing trainer.egg-info/PKG-INFO
writing dependency_links to trainer.egg-info/dependency_links.txt
writing requirements to trainer.egg-info/requires.txt
writing top-level names to trainer.egg-info/top_level.txt
writing manifest file 'trainer.egg-info/SOURCES.txt'
reading manifest file 'trainer.egg-info/SOURCES.txt'
writing manifest file 'trainer.egg-info/SOURCES.txt'
running check


creating trainer-0.1
creating trainer-0.1/trainer
creating trainer-0.1/trainer.egg-info
copying files to trainer-0.1...
copying README.md -> trainer-0.1
copying setup.py -> trainer-0.1
copying trainer/__init__.py -> trainer-0.1/trainer
copying trainer/experiment.py -> trainer-0.1/trainer
copying trainer/metadata.py -> trainer-0.1/trainer
copying trainer/model.py -> trainer-0.1/trainer
copying trainer/task.py -> trainer-0.1/trainer
copying trainer/utils.py -> trainer-0.1/trainer
copying trainer.egg-info/PKG-INFO -> trainer-0.1/trainer.egg-info
copying

In [ ]:
!gsutil cp {source_package_file_name} {python_package_gcs_uri}

Copying file://python_package/dist/trainer-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  5.8 KiB/  5.8 KiB]                                                
Operation completed over 1 objects/5.8 KiB.                                      


In [ ]:
!gsutil ls -l {python_package_gcs_uri}

      5971  2021-12-09T05:10:04Z  gs://starry-lens-333804aip-20211209030626/pytorch-on-gcp/finetuned-bert-classifier/train/python_package/trainer-0.1.tar.gz
TOTAL: 1 objects, 5971 bytes (5.83 KiB)


In [ ]:
!cd {APP_DIR} && python -m trainer.task

Namespace(batch_size=16, hp_tune='n', job_dir=None, learning_rate=2e-05, model_name='finetuned-bert-classifier', num_epochs=1, seed=42, weight_decay=0.01)
Using custom data configuration default
Reusing dataset emotion (/home/jupyter/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)
100%|█████████████████████████████████████████████| 2/2 [00:05<00:00,  2.76s/ba]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e

## Running custom training job on Vertex AI

We are using Vertex SDK for Python to create and submit training job to the Vertex pipeline.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BCT_NAME)

In [ ]:
print(f"APP_NAME={APP_NAME}")
print(
    f"PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI={TRAIN_URI}"
)
print(f"python_package_gcs_uri={python_package_gcs_uri}")
print(f"python_module_name={python_module_name}")

APP_NAME=finetuned-bert-classifier
PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-7:latest
python_package_gcs_uri=gs://starry-lens-333804aip-20211209030626/pytorch-on-gcp/finetuned-bert-classifier/train/python_package/trainer-0.1.tar.gz
python_module_name=trainer.task


In [ ]:
JOB_NAME = f"{APP_NAME}-pytorch-pkg-ar-{get_timestamp()}"
print(f"JOB_NAME={JOB_NAME}")

JOB_NAME=finetuned-bert-classifier-pytorch-pkg-ar-20211209051507


In [ ]:
jb = aiplatform.CustomPythonPackageTrainingJob(
    display_name=f"{JOB_NAME}",
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
)

In [ ]:
training_args = ["--num-epochs", "2", "--model-name", "finetuned-bert-classifier"]

model = jb.run(
    replica_count=1,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1,
    args=training_args,
    sync=False,
)

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-05:15:42.974 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8078420892037677056?project=425328907110
INFO:google.cloud.aiplatform.training_jobs:CustomPythonPackageTrainingJob projects/425328907110/locations/us-central1/trainingPipelines/8078420892037677056 current state:
PipelineState.PIPELINE_STATE_PENDING
INFO:google.cloud.aiplatform.training_jobs:CustomPythonPackageTrainingJob projects/425328907110/locations/us-central1/trainingPipelines/8078420892037677056 current state:
PipelineState.PIPELINE_STATE_PENDING
INFO:google.cloud.aiplatform.training_jobs:CustomPythonPackageTrainingJob projects/425328907110/locations/us-central1/trainingPipelines/8078420892037677056 current state:
PipelineState.PIPELINE_STATE_PENDING
INFO:google.cloud.aiplatform.training

## Validating the model artifacts written to GCS by the training code after the job is successful

In [ ]:

job_response = MessageToDict(job._gca_resource._pb)
gcs_model_artifacts_uri = job_response["trainingTaskInputs"]["baseOutputDirectory"][
    "outputUriPrefix"
]
print(f"Model artifacts are available at {gcs_model_artifacts_uri}")

Model artifacts are available at gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-05:15:42.974


In [ ]:
!gsutil ls -lr $gcs_model_artifacts_uri/

gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-05:15:42.974/:

gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-05:15:42.974/model/:

gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-05:15:42.974/model/finetuned-bert-classifier/:
       993  2021-12-09T05:33:58Z  gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-05:15:42.974/model/finetuned-bert-classifier/config.json
 433348873  2021-12-09T05:33:58Z  gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-05:15:42.974/model/finetuned-bert-classifier/pytorch_model.bin
       112  2021-12-09T05:33:58Z  gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-05:15:42.974/model/finetuned-bert-classifier/special_tokens_map.json
    435816  2021-12-09T05:33:58Z  gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-05:15:42.974/model/finetuned-b

In [ ]:
!gcloud container images describe $CUSTOM_TRAIN_IMAGE_URI


ERROR: (gcloud.container.images.describe) argument IMAGE_NAME: Must be specified.
Usage: gcloud container images describe IMAGE_NAME [optional flags]
  optional flags may be  --help

For detailed information on this command and its flags, run:
  gcloud container images describe --help


## Running a Custom Job on Vertex Training with docker

In [ ]:
%%writefile ./custom_container/Dockerfile

# Use pytorch GPU base image
FROM gcr.io/cloud-aiplatform/training/pytorch-gpu.1-7

# set working directory
WORKDIR /app

# Install required packages
RUN pip install google-cloud-storage transformers datasets tqdm cloudml-hypertune

# Copies the trainer code to the docker image.
COPY ./trainer/__init__.py /app/trainer/__init__.py
COPY ./trainer/experiment.py /app/trainer/experiment.py
COPY ./trainer/utils.py /app/trainer/utils.py
COPY ./trainer/metadata.py /app/trainer/metadata.py
COPY ./trainer/model.py /app/trainer/model.py
COPY ./trainer/task.py /app/trainer/task.py

# Set up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Overwriting ./custom_container/Dockerfile


In [ ]:
TRAIN_IMAGE = f"gcr.io/{PROJECT_ID}/pytorch_gpu_train_{APP_NAME}"


In [ ]:
!cd ./custom_container/ && docker build -f Dockerfile -t $TRAIN_IMAGE ../python_package

Sending build context to Docker daemon  56.37kB
Step 1/10 : FROM gcr.io/cloud-aiplatform/training/pytorch-gpu.1-7
latest: Pulling from cloud-aiplatform/training/pytorch-gpu.1-7

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
278deddf: Pulling fs layer 
80049843: Pulling fs layer 
556b2329: Pulling fs layer 
a0c97a55: Pulling fs layer 
78bd0b24: Pulling fs layer 
6c31766d: Pulling fs layer 
39245b99: Pulling fs layer 
5479a093: Pulling fs layer 
5ca4da05: Pulling fs layer 
fd6e9dc5: Pulling fs layer 
bbe64fe2: Pulling fs layer 
a27fb552: Pulling fs layer 
ecec6186: Pulling fs layer 
cb91ab9d: Pulling fs layer 
59d82e20: Pulling fs layer 
ccae33a4: Pulling fs layer 
6879b67d: Pulling fs layer 
0a6966fa: Pulling fs layer 
44eef141: Pulling fs layer 
9633c8ef: Pulling fs layer 
c3344851: Pulling fs layer 
4bc8487a: Pulling fs layer 
2a518419: Pulling fs layer 
d2a1ad5c: Pulling fs layer 
e5d4f578: Pulling fs layer 
4022ba47: Pulling fs layer 
2c6aa85d: 

## Pushing the container to Container Registry
Belo code helps us in pushing our container image with training code and dependencies to the Container Registry.

In [ ]:
!docker push $TRAIN_IMAGE

Using default tag: latest
The push refers to repository [gcr.io/starry-lens-333804/pytorch_gpu_train_finetuned-bert-classifier]

afbb97e0: Preparing 
108335c6: Preparing 
3bbb2426: Preparing 
9c05b48b: Preparing 
9465adca: Preparing 
08363895: Preparing 
d61e1dc6: Preparing 
419e0a82: Preparing 
6c56be71: Preparing 
105d38de: Preparing 
aacf220c: Preparing 
eb8da3b6: Preparing 
976e90b0: Preparing 
b42f0dc3: Preparing 
86a69e4c: Preparing 
bf027b10: Preparing 
53d15bde: Preparing 
667fdca8: Preparing 
20d6a257: Preparing 
7f0ad120: Preparing 
541dfbc1: Preparing 
a967deb1: Preparing 
e620d847: Preparing 
eceb9554: Preparing 
bb61b87c: Preparing 
9c9eea04: Preparing 
06241986: Preparing 
a36fa8ed: Preparing 
9e6952e1: Preparing 
af6a03f9: Preparing 
0507df07: Preparing 
3f825d1f: Preparing 
39619b27: Preparing 
1ad956a3: Preparing 
7c9f2e05: Preparing 
37ca5432: Preparing 
bc973851: Preparing 
f53ba690: Preparing 
d71984bd: Preparing 
014ab4a9: Preparing 
10271917: Preparing 
00c31be3: 

In [ ]:
##Validating the custom container image in Container Registry

!gcloud container images describe $TRAIN_IMAGE

image_summary:
  digest: sha256:3e360f75328589649ea52b55114057e2705cdecb377717eb1d311905501e9285
  fully_qualified_digest: gcr.io/starry-lens-333804/pytorch_gpu_train_finetuned-bert-classifier@sha256:3e360f75328589649ea52b55114057e2705cdecb377717eb1d311905501e9285
  registry: gcr.io
  repository: starry-lens-333804/pytorch_gpu_train_finetuned-bert-classifier


## Initialize the Vertex SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BCT_NAME)

## Configuring and submitting the Custom Job to Vertex Training pipeline
We are Configuring a Custom Job with custom container image created with training code and other dependencies

In [ ]:
JOB_NAME = f"{APP_NAME}-pytorch-cstm-cntr-{get_timestamp()}"

print(f"APP_NAME={APP_NAME}")
print(f"CUSTOM_TRAIN_IMAGE_URI={TRAIN_IMAGE}")
print(f"JOB_NAME={JOB_NAME}")

APP_NAME=finetuned-bert-classifier
CUSTOM_TRAIN_IMAGE_URI=gcr.io/starry-lens-333804/pytorch_gpu_train_finetuned-bert-classifier
JOB_NAME=finetuned-bert-classifier-pytorch-cstm-cntr-20211209060520


In [ ]:
# configure the job with container image spec
jb = aiplatform.CustomContainerTrainingJob(
    display_name=f"{JOB_NAME}", container_uri=f"{TRAIN_IMAGE}"
)

In [ ]:
# define training code arguments
training_args = ["--num-epochs", "2", "--model-name", "finetuned-bert-classifier"]

In [ ]:
# submit the custom job to Vertex training service
mdl = job.run(
    replica_count=1,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1,
    args=training_args,
    sync=False,
)

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://starry-lens-333804aip-20211209030626/aiplatform-custom-training-2021-12-09-06:06:04.175 


In [ ]:
!gcloud container images describe $CUSTOM_TRAIN_IMAGE_URI

image_summary:
  digest: sha256:3e360f75328589649ea52b55114057e2705cdecb377717eb1d311905501e9285
  fully_qualified_digest: gcr.io/starry-lens-333804/pytorch_gpu_train_finetuned-bert-classifier@sha256:3e360f75328589649ea52b55114057e2705cdecb377717eb1d311905501e9285
  registry: gcr.io
  repository: starry-lens-333804/pytorch_gpu_train_finetuned-bert-classifier


## Running Hyperparameter Tuning Job on Vertex AI
We are experimenting with hyperparameters such as learning rate and weight decay while fine tuning the BERT model. These hyperparameter values are directly proportional to the behavior of the training algorithm and can have a significant impact on the performance of the model. We have automated the hyperparameter tuning with the Vertex Training service by packaging the training code and all dependencies in a Docker container. We then pushed the container to Google Container Registry.


In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)


### Configuring and submitting Hyperparameter Tuning Job to Vertex Training pipeline

In [ ]:
JOB_NAME = f"{APP_NAME}-pytorch-hptune-{get_timestamp()}"

print(f"APP_NAME={APP_NAME}")
print(f"CUSTOM_TRAIN_IMAGE_URI={CUSTOM_TRAIN_IMAGE_URI}")
print(f"JOB_NAME={JOB_NAME}")

APP_NAME=finetuned-bert-classifier
CUSTOM_TRAIN_IMAGE_URI=gcr.io/starry-lens-333804/pytorch_gpu_train_finetuned-bert-classifier
JOB_NAME=finetuned-bert-classifier-pytorch-hptune-20211209071625


In [ ]:
training_args = [
    "--num-epochs",
    "2",
    "--model-name",
    "finetuned-bert-classifier",
    "--hp-tune",
    "y",
]

In [ ]:
# The spec of the worker pools including machine type and Docker image
pool_specialities = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-8",
            "accelerator_type": "NVIDIA_TESLA_V100",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {"image_uri": CUSTOM_TRAIN_IMAGE_URI, "args": training_args},
    }
]

In [ ]:
custom_job = aiplatform.CustomJob(
    display_name=JOB_NAME, worker_pool_specialities=worker_pool_specs
)

In [ ]:
# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
prmtr_spec = {
    "learning-rate": hpt.DoubleParameterSpec(min=1e-6, max=0.001, scale="log"),
    "weight-decay": hpt.DiscreteParameterSpec(
        values=[0.0001, 0.001, 0.01, 0.1], scale=None
    ),
}

In [ ]:
# Dictionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec = {"accuracy": "maximize"}

In [ ]:
JOB_NAME = "finetuned-bert-classifier-pytorch-hptune-20211209072805"

In [ ]:
hyper_job = aiplatform.HyperparameterTuningJob(
    display_name=JOB_NAME,
    custom_job=custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=5,
    parallel_trial_count=2,
    search_algorithm=None,
)

In [ ]:
mdl = hp_job.run(sync=False)

INFO:google.cloud.aiplatform.jobs:Creating HyperparameterTuningJob


### Running docker container custom job locally

In [ ]:
!docker run --gpus all -it --rm $CUSTOM_TRAIN_IMAGE_URI

Namespace(batch_size=16, hp_tune='n', job_dir=None, learning_rate=2e-05, model_name='finetuned-bert-classifier', num_epochs=1, seed=42, weight_decay=0.01)
Downloading: 3.62kB [00:00, 3.19MB/s]                                           
Downloading: 3.28kB [00:00, 3.26MB/s]                                           
Using custom data configuration default
Downloading: 100%|█████████████████████████| 1.66M/1.66M [00:00<00:00, 9.18MB/s]
Downloading: 100%|███████████████████████████| 204k/204k [00:00<00:00, 2.52MB/s]
Downloading: 100%|███████████████████████████| 207k/207k [00:00<00:00, 2.14MB/s]
Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.
  0%|                                                    | 0/16 [00:00<?, ?ba/s]
Downloading: 100%|███████████████████████████| 29.0/29.0 [00:00<00:00, 32.0kB/s]

Downloading: 100%|███████████████

In [ ]:
!cd custom_container && ./scripts/train-cloud.sh

Submitting PyTorch model training job to Vertex AI
Sending build context to Docker daemon  56.48kB
Step 1/10 : FROM gcr.io/cloud-aiplatform/training/pytorch-gpu.1-7
 ---> 1d793c644e7e
Step 2/10 : WORKDIR /app
 ---> Running in 0b3086cd35f7
Removing intermediate container 0b3086cd35f7
 ---> 92d0abbdc520
Step 3/10 : RUN pip install google-cloud-storage transformers datasets tqdm cloudml-hypertune
 ---> Running in 8b693b1f1a40
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.8.7
    Uninstalling fsspec-0.8.7:
      Successfully uninstalled fsspec-0.8.7
Removing intermediate container 8b693b1f1a40
 ---> 437f1b78d705
Step 4/10 : COPY ./trainer/__init__.py /app/trainer/__init__.py
 ---> e5b3914cc84c
Step 5/10 : COPY ./trainer/experiment.py /app/trainer/experiment.py
 ---> 08dc7020db8c
Step 6/10 : COPY ./trainer/utils.py /app

## Alternative way to submit hyperparameter job to Vertex AI

In [ ]:
%%bash -s $BUCKET_NAME $APP_NAME

# ========================================================
# set job parameters
# ========================================================
# PROJECT_ID: Change to your project id
PRJCT_ID=$(gcloud config list --format 'value(core.project)')

# set job display name
jb_pREFIX="finetuned-bert-classifier"
JB_NAME=${JOB_PREFIX}-pytorch-hptune-$(date +%Y%m%d%H%M%S)
echo "Launching hyperparameter tuning job with display name as "$JOB_NAME

# BUCKET_NAME: Change to your bucket name
BCT_NAME=$1 # <-- CHANGE TO YOUR BUCKET NAME

# APP_NAME: get application name
APP_NAME=$2

# JOB_DIR: Where to store prepared package and upload output model.
JOB_DIR=${BUCKET_NAME}/${JOB_PREFIX}/model/${JOB_NAME}

# custom container image URI
TRAIN_IMAGE="gcr.io/starry-lens-333804/pytorch_gpu_train_finetuned-bert-classifier"
# CUSTOM_TRAIN_IMAGE_URI=f'gcr.io/'${PROJECT_ID}'/pytorch_gpu_train_'${APP_NAME}

# ========================================================
# create hyperparameter tuning configuration file
# ========================================================
cat << EOF > ./python_package/hptuning_job.yaml

studySpec:
  metrics:
  - metricId: accuracy
    goal: MAXIMIZE
  parameters:
  - parameterId: learning-rate
    scaleType: UNIT_LOG_SCALE
    doubleValueSpec:
      minValue: 0.000001
      maxValue: 0.001
  - parameterId: weight-decay
    scaleType: SCALE_TYPE_UNSPECIFIED
    discreteValueSpec:
      values: [
          0.0001,
          0.001,
          0.01,
          0.1
      ]
  measurementSelectionType: BEST_MEASUREMENT
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: n1-standard-8
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1
    replicaCount: 1
    containerSpec:
      imageUri: $CUSTOM_TRAIN_IMAGE_URI
      args: ["--num-epochs", "2", "--model-name", "finetuned-bert-classifier", "--hp-tune", "y"]
  baseOutputDirectory:
    outputUriPrefix: $JOB_DIR/
EOF

# ========================================================
# submit hyperparameter tuning job
# ========================================================
gcloud beta ai hp-tuning-jobs create \
   --config ./python_package/hptuning_job.yaml \
   --display-name $JOB_NAME \
   --algorithm algorithm-unspecified \
   --max-trial-count 5 \
   --parallel-trial-count 2 \
   --region=us-west1

Launching hyperparameter tuning job with display name as finetuned-bert-classifier-pytorch-hptune-20211209072805


Using endpoint [https://us-west1-aiplatform.googleapis.com/]
Hyperparameter tuning job [919807447332290560] submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud beta ai hp-tuning-jobs describe 919807447332290560 --region=us-west1

Job State: JOB_STATE_PENDING


### Getting the best model after hyperparameter tuning

In [ ]:
def get_as_df(trials):
    results = []
    for trial in trials:
        row = {}
        t = MessageToDict(trial._pb)
        # print(t)
        row["Trial ID"], row["Status"], row["Start time"], row["End time"] = (
            t["id"],
            t["state"],
            t["startTime"],
            t.get("endTime", None),
        )

        for param in t["parameters"]:
            row[param["parameterId"]] = param["value"]

        if t["state"] == "SUCCEEDED":
            row["Training step"] = t["finalMeasurement"]["stepCount"]
            for metric in t["finalMeasurement"]["metrics"]:
                row[metric["metricId"]] = metric["value"]
        results.append(row)

    _df = pd.DataFrame(results)
    return _df

In [ ]:
!gsutil ls -r $best_model_artifact_uri

## Deployment
### Defining a custom text handler to preprocess and postprocess data

In [ ]:


import os
import json
import logging

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from ts.torch_handler.base_handler import BaseHandler

logger = logging.getLogger(__name__)


class TransformersClassifierHandler(BaseHandler):
    """
    The handler takes an input string and returns the classification text
    based on the serialized transformers checkpoint.
    """
    def __init__(self):
        super(TransformersClassifierHandler, self).__init__()
        self.initialized = False

    def initialize(self, ctx):
        """ Loads the model.pt file and initialized the model object.
        Instantiates Tokenizer for preprocessor to use
        Loads labels to name mapping file for post-processing inference response
        """
        self.manifest = ctx.manifest

        properties = ctx.system_properties
        model_dir = properties.get("model_dir")
        self.device = torch.device("cuda:" + str(properties.get("gpu_id")) if torch.cuda.is_available() else "cpu")

        # Read model serialize/pt file
        serialized_file = self.manifest["model"]["serializedFile"]
        model_pt_path = os.path.join(model_dir, serialized_file)
        if not os.path.isfile(model_pt_path):
            raise RuntimeError("Missing the model.pt or pytorch_model.bin file")

        # Load model
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        self.model.to(self.device)
        self.model.eval()
        logger.debug('Transformer model from path {0} loaded successfully'.format(model_dir))

        # Ensure to use the same tokenizer used during training
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

        # Read the mapping file, index to object name
        mapping_file_path = os.path.join(model_dir, "index_to_name.json")

        if os.path.isfile(mapping_file_path):
            with open(mapping_file_path) as f:
                self.mapping = json.load(f)
        else:
            logger.warning('Missing the index_to_name.json file. Inference output will not include class name.')

        self.initialized = True

    def preprocess(self, data):
        """ Preprocessing input request by tokenizing
            Extend with your own preprocessing steps as needed
        """
        text = data[0].get("data")
        if text is None:
            text = data[0].get("body")
        sentences = text.decode('utf-8')
        logger.info("Received text: '%s'", sentences)

        # Tokenize the texts
        tokenizer_args = ((sentences,))
        inputs = self.tokenizer(*tokenizer_args,
                                padding='max_length',
                                max_length=128,
                                truncation=True,
                                return_tensors = "pt")
        return inputs

    def inference(self, inputs):
        """ Predict the class of a text using a trained transformer model.
        """
        prediction = self.model(inputs['input_ids'].to(self.device))[0].argmax().item()

        if self.mapping:
            prediction = self.mapping[str(prediction)]

        logger.info("Model predicted: '%s'", prediction)
        return [prediction]

    def postprocess(self, inference_output):
        return inference_output

Overwriting predictor/custom_text_handler.py


In [ ]:


{
    "0": "Sadness",
    "1": "Joy",
    "2": "love",
    "3": "anger",
    "4": "fear"
}

Overwriting ./predictor/index_to_name.json


In [ ]:
GCS_MODEL_ARTIFACTS_URI = best_model_artifact_uri


In [ ]:
!gsutil ls -r $GCS_MODEL_ARTIFACTS_URI/model/

gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/:

gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/finetuned-bert-classifier/:
gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/finetuned-bert-classifier/config.json
gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/finetuned-bert-classifier/pytorch_model.bin
gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/finetuned-bert-classifier/special_tokens_map.json
gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/finetuned

In [ ]:
!gsutil -m cp -r $GCS_MODEL_ARTIFACTS_URI/model/ ./predictor/

Copying gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/finetuned-bert-classifier/tokenizer.json...
Copying gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/finetuned-bert-classifier/config.json...
Copying gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/finetuned-bert-classifier/pytorch_model.bin...
Copying gs://starry-lens-333804aip-20211209030626/finetuned-bert-classifier/model/finetuned-bert-classifier-pytorch-hptune-20211209072805/5/model/finetuned-bert-classifier/special_tokens_map.json...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (se

In [ ]:
!ls -ltrR ./predictor/model

./predictor/model:
total 4
drwxr-xr-x 2 jupyter jupyter 4096 Dec  9 08:55 finetuned-bert-classifier

./predictor/model/finetuned-bert-classifier:
total 423852
-rw-r--r-- 1 jupyter jupyter       993 Dec  9 08:55 config.json
-rw-r--r-- 1 jupyter jupyter       112 Dec  9 08:55 special_tokens_map.json
-rw-r--r-- 1 jupyter jupyter       320 Dec  9 08:55 tokenizer_config.json
-rw-r--r-- 1 jupyter jupyter    213450 Dec  9 08:55 vocab.txt
-rw-r--r-- 1 jupyter jupyter    435816 Dec  9 08:55 tokenizer.json
-rw-r--r-- 1 jupyter jupyter      2863 Dec  9 08:55 training_args.bin
-rw-r--r-- 1 jupyter jupyter 433348873 Dec  9 08:55 pytorch_model.bin


## Building a custom docker container with the best model

In [ ]:
%%bash -s $APP_NAME

APP_NAME=$1

cat << EOF > ./predictor/Dockerfile

FROM pytorch/torchserve:latest-cpu

# install dependencies
RUN pip3 install transformers

# copy model artifacts, custom handler and other dependencies
COPY ./custom_text_handler.py /home/model-server/
COPY ./index_to_name.json /home/model-server/
COPY ./model/$APP_NAME/ /home/model-server/

# create torchserve configuration file
USER root
RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/config.properties
RUN printf "\nmanagement_address=http://0.0.0.0:7081" >> /home/model-server/config.properties
USER model-server

# expose health and prediction listener ports from the image
EXPOSE 7080
EXPOSE 7081

# create model archive file packaging model artifacts and dependencies
RUN torch-model-archiver -f \
  --model-name=$APP_NAME \
  --version=1.0 \
  --serialized-file=/home/model-server/pytorch_model.bin \
  --handler=/home/model-server/custom_text_handler.py \
  --extra-files "/home/model-server/config.json,/home/model-server/tokenizer.json,/home/model-server/training_args.bin,/home/model-server/tokenizer_config.json,/home/model-server/special_tokens_map.json,/home/model-server/vocab.txt,/home/model-server/index_to_name.json" \
  --export-path=/home/model-server/model-store

# run Torchserve HTTP serve to respond to prediction requests
CMD ["torchserve", \
     "--start", \
     "--ts-config=/home/model-server/config.properties", \
     "--models", \
     "$APP_NAME=$APP_NAME.mar", \
     "--model-store", \
     "/home/model-server/model-store"]
EOF

echo "Writing ./predictor/Dockerfile"

Writing ./predictor/Dockerfile


In [ ]:
PREDICTOR_IMAGE = f"gcr.io/{PROJECT_ID}/pytorch_predict_{APP_NAME}"
print(f"CUSTOM_PREDICTOR_IMAGE_URI = {CUSTOM_PREDICTOR_IMAGE_URI}")

CUSTOM_PREDICTOR_IMAGE_URI = gcr.io/starry-lens-333804/pytorch_predict_finetuned-bert-classifier


In [ ]:
!docker build \
  --tag=$CUSTOM_PREDICTOR_IMAGE_URI \
  ./predictor

Sending build context to Docker daemon    434MB
Step 1/14 : FROM pytorch/torchserve:latest-cpu
latest-cpu: Pulling from pytorch/torchserve

d2c87b75: Pulling fs layer 
10be24e1: Pulling fs layer 
7173dcfe: Pulling fs layer 
d40b7ebd: Pulling fs layer 
7c5679f0: Pulling fs layer 
1d21c9ce: Pulling fs layer 
d91f5b58: Pulling fs layer 
2cd6c42d: Pulling fs layer 
03a87009: Pulling fs layer 
cf069356: Pulling fs layer 
b700ef54: Pull complete   32B/32BB4kBB[11ADownloading  2.668MB/205.6MBDigest: sha256:31e04f199b5a070f53a519fe436eed9aacc55172891601e7224fd9c3ad95b2c2
Status: Downloaded newer image for pytorch/torchserve:latest-cpu
 ---> 659d9f4840d5
Step 2/14 : RUN pip3 install transformers
 ---> Running in a9e3e86087bb
Removing intermediate container a9e3e86087bb
 ---> adc3973b0f91
Step 3/14 : COPY ./custom_text_handler.py /home/model-server/
 ---> 7a3c72fb313a
Step 4/14 : COPY ./index_to_name.json /home/model-server/
 ---> 97c4a98d5d9f
Step 5/14 : COPY ./model/finetuned-bert-classifier/ 

### Run the container locally

In [ ]:
!docker stop local_bert_classifier
!docker run -t -d --rm -p 7080:7080 --name=local_bert_classifier $CUSTOM_PREDICTOR_IMAGE_URI
!sleep 20

Error response from daemon: No such container: local_bert_classifier
fca2689ea970147c8e327b67f09c7f579a915b6704335db0004eb163ab0f9986


In [ ]:
!docker ps

CONTAINER ID   IMAGE                                                                 COMMAND                  CREATED          STATUS          PORTS                                                            NAMES
fca2689ea970   gcr.io/starry-lens-333804/pytorch_predict_finetuned-bert-classifier   "/usr/local/bin/dock…"   24 seconds ago   Up 23 seconds   7070-7071/tcp, 7081/tcp, 8080-8082/tcp, 0.0.0.0:7080->7080/tcp   local_bert_classifier
e55a61e001ed   gcr.io/inverting-proxy/agent                                          "/bin/sh -c '/opt/bi…"   6 hours ago      Up 6 hours                                                                       proxy-agent


In [ ]:
!curl http://localhost:7080/ping

{
  "status": "Healthy"
}


In [ ]:
%%bash -s $APP_NAME

APP_NAME=$1

cat > ./predictor/instances.json <<END
{
   "instances": [
     {
       "data": {
         "b64": "$(echo 'I love pizza' | base64 --wrap=0)"
       }
     }
   ]
}
END

curl -s -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @./predictor/instances.json \
  http://localhost:7080/predictions/$APP_NAME/

{"predictions": ["Joy"]}

## Deploying the serving container to Vertex AI

In [ ]:
!docker push $CUSTOM_PREDICTOR_IMAGE_URI

Using default tag: latest
The push refers to repository [gcr.io/starry-lens-333804/pytorch_predict_finetuned-bert-classifier]

7322eb6d: Preparing 
f8cd4e5e: Preparing 
004756de: Preparing 
855761fa: Preparing 
abed59b2: Preparing 
782ec13d: Preparing 
56320d30: Preparing 
58eacbff: Preparing 
bf18a086: Preparing 
d2b4edc6: Preparing 
4d0230ad: Preparing 
f03ffca8: Preparing 
f2874ea9: Preparing 
a1d7f3ba: Preparing 
d7c91a07: Preparing 
c3f5e5be: Preparing 
512fd434: Preparing 
31fc0e08: Preparing 
a1d7f3ba: Pushed   818.9MB/812.1MBPushing  182.1MB/516.4MBlatest: digest: sha256:221b726ac4b8943631532e974bd7323950de33fd2396de9ca4cd11c442192cd4 size: 4287


In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

### Creating a Model resource with custom serving container

In [ ]:
VERSION = 1
model_display_name = f"{APP_NAME}-v{VERSION}"
model_description = "PyTorch based text classifier with custom container"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = f"/predictions/{MODEL_NAME}"
serving_container_ports = [7080]

In [ ]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=CUSTOM_PREDICTOR_IMAGE_URI,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

model.wait()

print(model.display_name)
print(model.resource_name)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/425328907110/locations/us-central1/models/7271840052622131200/operations/6133184780105154560
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/425328907110/locations/us-central1/models/7271840052622131200
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/425328907110/locations/us-central1/models/7271840052622131200')
finetuned-bert-classifier-v1
projects/425328907110/locations/us-central1/models/7271840052622131200


In [ ]:
### Create an Endpoint for Model with Custom Container

In [ ]:
endpoint_display_name = f"{APP_NAME}-endpoint"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/425328907110/locations/us-central1/endpoints/2463548161008861184/operations/7923365631984926720
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/425328907110/locations/us-central1/endpoints/2463548161008861184
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/425328907110/locations/us-central1/endpoints/2463548161008861184')


In [ ]:
trffc_percentage = 100
mach_type = "n1-standard-4"
dpd_model_display_name = model_display_name
mn_replica_count = 1
mx_replica_count = 3
sync = True

model.deploy(
    endpoint=endpoint,
    dpd_model_display_name=deployed_model_display_name,
    mach_type=machine_type,
    trffc_percentage=traffic_percentage,
    sync=sync,
)

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/425328907110/locations/us-central1/endpoints/2463548161008861184
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/425328907110/locations/us-central1/endpoints/2463548161008861184/operations/3832830532937318400
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/425328907110/locations/us-central1/endpoints/2463548161008861184


resource name: projects/425328907110/locations/us-central1/endpoints/2463548161008861184

In [ ]:
endpoint_name = f"{APP_NAME}-endpoint"
fltr = f'display_name="{endpoint_name}"'

for endpoint_info in aiplatform.Endpoint.list(filter=fltr):
    print(
        f"Endpoint display name = {endpoint_info.display_name} resource id ={endpoint_info.resource_name} "
    )

endpoint = aiplatform.Endpoint(endpoint_info.resource_name)

Endpoint display name = finetuned-bert-classifier-endpoint resource id =projects/425328907110/locations/us-central1/endpoints/2463548161008861184 


In [ ]:
endpoint.list_models()

[id: "2249662363080851456"
model: "projects/425328907110/locations/us-central1/models/7271840052622131200"
display_name: "finetuned-bert-classifier-v1"
create_time {
  seconds: 1639090350
  nanos: 175393000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

In [ ]:
tstt_instances = [
    b"Jaw dropping visual affects and action! One of the best I have seen to date.",
    b"Take away the CGI and the A-list cast and you end up with film with less punch.",
]

In [ ]:
print("=" * 100)
for instance in test_instances:
    print(f"Input text: \n\t{instance.decode('utf-8')}\n")
    b64_encoded = base64.b64encode(instance)
    test_instance = [{"data": {"b64": f"{str(b64_encoded.decode('utf-8'))}"}}]
    print(f"Formatted input: \n{json.dumps(tst_instance, indent=4)}\n")
    prediction = endpoint.predict(instances=tst_instance)
    print(f"Prediction response: \n\t{prediction}")
    print("=" * 100)

Input text: 
	Jaw dropping visual affects and action! One of the best I have seen to date.

Formatted input: 
[
    {
        "data": {
            "b64": "SmF3IGRyb3BwaW5nIHZpc3VhbCBhZmZlY3RzIGFuZCBhY3Rpb24hIE9uZSBvZiB0aGUgYmVzdCBJIGhhdmUgc2VlbiB0byBkYXRlLg=="
        }
    }
]

Prediction response: 
	Prediction(predictions=['Joy'], deployed_model_id='2249662363080851456', explanations=None)
Input text: 
	Take away the CGI and the A-list cast and you end up with film with less punch.

Formatted input: 
[
    {
        "data": {
            "b64": "VGFrZSBhd2F5IHRoZSBDR0kgYW5kIHRoZSBBLWxpc3QgY2FzdCBhbmQgeW91IGVuZCB1cCB3aXRoIGZpbG0gd2l0aCBsZXNzIHB1bmNoLg=="
        }
    }
]

Prediction response: 
	Prediction(predictions=['Joy'], deployed_model_id='2249662363080851456', explanations=None)
